# [George McNinch](http://gmcninch.math.tufts.edu) Math 87 - Spring 2024

# Week 13

# Spring networks


# Modeling spring networks

Examples of "Structural modeling" that will ultimately involve *least squares* approximation.

## One-dimensional models

Let's consider a linear network of 3 springs and 2 masses:

![](../course-assets/images/springs--image01.png)

Here are the variables:

- $f_j$ = applied load or force to mass (in `N` = `Newtons`), for $j=1,2$
- $k_i$ = spring constant (in `N/m` = `Newtons per meter`), for $i=1,2,3$ 
- $e_i$ = elongation of spring $i$ from equilibrium (in `m` = `meters`)
- $x_j$ = displasement of mass $j$ from equilibrium (in `m` = `meters`)
- $y_i$ = restoring force on spring $i$ (in `N` = `Newtons`)

The *"inputs"* are the applied forces $f_j$ which cause the masses to move, resulting in elongation of springs.

We'll take "movement to the right" to be *positive*, and a stretch as *positive* elongation.

Thus we have the equations:
$$e_1 = x_1, \quad e_2 = x_2 - x_1, \quad e_3 = -x_2.$$

(This third equation reflects the fact that spring 3 compresses when $m_2$ moves to the right.)

Let's put this in matrix form:

$$\mathbf{e} = \begin{bmatrix} e_1 \\ e_2 \\ e_3 \end{bmatrix} =
\begin{bmatrix} 1 & 0 \\ -1 & 1 \\ 0 & -1 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \end{bmatrix}
= B \mathbf{x}.$$

Now, let's recall that according to [Hooke's Law](https://en.wikipedia.org/wiki/Hooke's_law),
the elongation of the spring causes a restoring force on the mass, determined by the *spring constant* $k_i>0$. Thus we get equations

$$y_j = k_j e_j \quad \text{for $j = 1,2,3$}.$$

In matrix form, these equations read:

$$\mathbf{y} = \begin{bmatrix} y_1 \\ y_2 \\ y_3 \end{bmatrix}
= \begin{bmatrix}  k_1 & 0 & 0 \\ 0 & k_2 & 0 \\ 0 & 0 & k_3 \end{bmatrix}
\begin{bmatrix} e_1 \\ e_2 \\ e_3 \end{bmatrix} = K\mathbf{e}.$$

*Combining* these euqations gives
$$\mathbf{y} = K \mathbf{e} = K B \mathbf{x}.$$

In [15]:
import numpy as np
import numpy.linalg as la

import sympy as sp

def sbv(i,n):
    return np.array([1 if j == i else 0 for j in range(n)])

def makeK(kar):
    n = len(kar)
    return np.array([kar[i]*sbv(i,n) for i in range(n)])

def makeB(m,n):
    return np.array([(-1)*sbv(i-1,m) + sbv(i,m) for i in range(n)])

k1, k2, k3 = sp.symbols('k1 k2 k3')

K = makeK([k1,k2,k3])
B = makeB(2,3)
(K,B)

(array([[k1, 0, 0],
        [0, k2, 0],
        [0, 0, k3]], dtype=object),
 array([[ 1,  0],
        [-1,  1],
        [ 0, -1]]))


Next, we assume that the system is at rest after the loads are applied (i.e. the forces $f_i$).

![](../course-assets/images/springs--image02.png)

Looking at the diagram, we see that the following equations must hold:

(The first diagram gives:)
\begin{align*}
y_1 &= y_2 + f_1 \implies\\
y_1 - y_2 & = f_1 
\end{align*}

(The second diagram gives:)
\begin{align*}
y_2 &= y_3 + f_2 \implies\\
y_2 - y_3 &= f_2
\end{align*}

In matrix form this reads
$$\begin{bmatrix}
1 & -1 & 0 \\
0 & 1 & -1
\end{bmatrix}
\begin{bmatrix} y_1 \\ y_2 \\ y_3 \end{bmatrix} =
\begin{bmatrix} f_1 \\ f_2 \end{bmatrix},$$
i.e.
$$B^T \mathbf{y} = \mathbf{f}$$

Combined with our earlier equation
$$\mathbf{y} = K \mathbf{e} = K B \mathbf{x}$$
we now see
$$B^T K B \mathbf{x} = \mathbf{f}.$$

In [2]:
A=B.transpose() @ K @ B 
A

array([[k1 + k2, -k2],
       [-k2, k2 + k3]], dtype=object)

Thus we have
$$A = B^T K B = \begin{bmatrix} k_1 + k_2 & -k_2 \\
-k_2 & k_2 + k_3 \end{bmatrix}.$$

In [16]:
def findDisplacements(kar,far):
    # kar = array of spring constants
    # far = array of initial forces.
    m = len(far)
    n = len(kar)
    B = makeB(m,n)
    K = makeK(kar)
    A = B.transpose() @ K @ B
    f = np.array(far)
    return la.solve(A,f)

# Let's find the displacements for spring constants `k = [1,1,1]`
# and forces `f = [3,-3]`

findDisplacements([1,1,1],[3,-3])

array([ 1., -1.])

In [4]:
findDisplacements([1,1,1],[3,-2])

array([ 1.33333333, -0.33333333])

## Two dimensional models

Now let's allow the mass to move in two dimensions:

![](../course-assets/images/springs--image03.png)

We see in this case that the elongation $e$ satisfies

$$e = \sqrt{x_1^2 + (L + x_2)^2} - L$$

Since this does not express a linear relationship between $e$ and the displacements
$x_1,x_2$, we can't express this relationship using a matrix.

But we can *linearize*. Recall that the linearization (first-order Taylor polynomial) about
$t=0$ of the function $y = \sqrt{1+t}$ is given by
$$(\clubsuit) \quad \sqrt{1+t} \approx 1 + \dfrac{t}{2} + O(t^2).$$

Let's use this linearization to rewrite the expression for $e$ given above.

We first rewrite
\begin{align*}
x_1^2 + (L+ x_2)^2 &= x_1^2 + L^2 + 2Lx_2 + x_2^2 \\
&= L^2\left( \dfrac{x_1^2}{L^2} + 1 + \dfrac{2x_2}{L} + \dfrac{x_2^2}{L^2} \right) \\
&= L^2\left( 1 + \dfrac{2x_2}{L} + \dfrac{x_1^2}{L^2} + \dfrac{x_2^2}{L^2} \right) 
\end{align*}
so that
\begin{align*}
e &= \sqrt{x_1^2 + (L + x_2)^2} - L \\
&= \sqrt{L^2\left( 1 + \dfrac{2x_2}{L} + \dfrac{x_1^2}{L^2} + \dfrac{x_2^2}{L^2} \right)} -L \\
&= L\sqrt{1 + \dfrac{2x_2}{L} + \dfrac{x_1^2}{L^2} + \dfrac{x_2^2}{L^2}} -L
\end{align*}

Now taking $t = \dfrac{2x_2}{L} + \dfrac{x_1^2}{L^2} + \dfrac{x_2^2}{L^2}$ the approximation
$(\clubsuit)$ gives
\begin{align*}
e &\approx L\left(1 + \dfrac{1}{2}t\right) - L \\
&= L \left(1 + \dfrac{1}{2}\dfrac{2x_2}{L} + \dfrac{x_1^2}{L^2} + \dfrac{x_2^2}{L^2}\right) -L \\
&= x_2 + \dfrac{x_1^2 + x_2^2}{2L}
\end{align*}

Now, this is of course still not a linear relationship between $e$ and $x_1,x_2$. Note that the approximation $(\clubsuit)$ depends on the assumption that $t = \dfrac{2x_2}{L} + \dfrac{x_1^2}{L^2} + \dfrac{x_2^2}{L^2} \approx 0$.

If we suppose that the displacements $x_1,x_2$ are small compared to the resting length $L$ of the spring, then $x_1^2 + x_2^2$ is even smaller compared to $L$, so making one more approximation, we eliminate the quadratic term and so we get
$$e \approx x_2.$$

**Remark**
:   The approximation $e \approx x_2$ is equivalent to a [small-angle approximation](https://en.wikipedia.org/wiki/Small-angle_approximation). 
    It essentially says that the horizontal displacements are negligible in the elongation, but vertical displacements are important.

**Remark**
:   This assumption is of course **"wrong"**, but can still be useful. Especially, it is now *linear*

## Spring networks

Let's consider a  network of springs in the two-dimensional setting.

![](../course-assets/images/springs--image04.png)

The *linearization* discussed in the previous section says that the elongation
is determined by the displacement in the "1 dimensional direction of the spring".

We get the matrix equation

$$\mathbf{e} = \begin{bmatrix} 0 & 1 & 0 & 0 \\ -1 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \end{bmatrix}
\begin{bmatrix} x_1 \\ x_2 \\ x_3 \\ x_4 \end{bmatrix} = B \mathbf{x}.$$

i.e. $$B = \begin{bmatrix} 0 & 1 & 0 & 0 \\ -1 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \end{bmatrix}$$

As before, Hooke's Law gives

$$\mathbf{y} = \begin{bmatrix} y_1 \\ y_2 \\ y_3 \end{bmatrix}
= \begin{bmatrix}  k_1 & 0 & 0 \\ 0 & k_2 & 0 \\ 0 & 0 & k_3 \end{bmatrix}
\begin{bmatrix} e_1 \\ e_2 \\ e_3 \end{bmatrix} = K\mathbf{e}.$$


We see again that the equations to balance the forces come from the transposed matrix $B^T$:

![](../course-assets/images/springs--image05.png)

This gives

$$\begin{bmatrix} 0 & -1 & 0 \\
 1 & 0 & 0 \\
 0 & 1 & 0 \\
 0 & 0 & 1 \end{bmatrix} \mathbf{y} = B^T \mathbf{y}  = \mathbf{f}.$$

Thus
$$B^T K B \mathbf{x} = \mathbf{f}.$$

In [73]:
B = np.array([[0,1,0,0],[-1,0,1,0],[0,0,0,1]])

K=makeK([1,1,1])           ## springs of equal strength
f = np.array([0,1,0,1])    ## downward forces only 

A = B.transpose() @ K @ B

try:
   la.solve(A , f) 
except:
   print(f"the matrix B^T K B = \n\n{A}\n\n is singular")

the matrix B^T K B = 

[[ 1  0 -1  0]
 [ 0  1  0  0]
 [-1  0  1  0]
 [ 0  0  0  1]]

 is singular


In [69]:
# numpy confirms that the null space has dimension 1

la.matrix_rank(A)

3

In [74]:
# so we just need to find a non-zero vector in the null space

# now, the Null space of A can be obtained from the *singular value decomposition* of A.

# la.svd(A) returns a tuple ( U , S, Vh )
# where e.g. U is a unitary matrix

U,S,Vh = la.svd(A)

# notice that the last column of U is in the null space:

A@U

array([[-1.41421356,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 1.41421356,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ]])

In [75]:
nullA = U[:,3]
A @ nullA

array([0., 0., 0., 0.])

In [76]:
# now we just need a solution to A @ x = f
# let's use `lstsq` to get one

x,_,_,_ = la.lstsq(A,f,rcond=None)
(x,nullA)

(array([0., 1., 0., 1.]),
 array([0.70710678, 0.        , 0.70710678, 0.        ]))

Thus we see that the general solution to $$A \mathbf{x} = \mathbf{f}$$ is given by
$$x = \begin{bmatrix} 0 \\ 1 \\ 0 \\ 1 \end{bmatrix} + t \begin{bmatrix} 1 \\ 0 \\ 1 \\ 0 \end{bmatrix} = \begin{bmatrix} t \\ 1 \\ t \\ 1 \end{bmatrix}.$$

What is the physical meaning of the solutions given by the null space of $A$? i.e. the solutions
$$t\mathbf{x}_n = t \begin{bmatrix} 1 \\ 0 \\ 1 \\ 0 \end{bmatrix}$$

Well, since $A \mathbf{x}_n = \mathbf{0}$, we see that a non-zero displacement can result from application of *zero force* (!).

This is clearly "wrong", and is a consequence of our linearization, **but** 
these displacement are referred to as *unstable modes*, and they do correspond to some real physical phenomena -- 

see e.g. [Tacoma bridge collapse (1940)](https://www.youtube.com/watch?v=XggxeuFDaDU)

# Stabilizing the unstable modes

One way to try to stabilize the unstable modes is to add a diagonal spring:

![](../course-assets/images/springs--image06.png)

For the indicate angle $\theta$, the elongation is given by
$$e  = \sqrt{(L\cos(\theta) + x_1)^2 + (L \sin(\theta) + x_2)^2}$$

Now our linearization assumption gives
$$e \approx x_1 \cos(\theta) + x_2 \sin(\theta).$$

Observe that under the linearization, the elongation is again in the "spring direction". 

Note e.g. if $\theta = 0$ then $e = x_1$ and if $\theta = \dfrac{\pi}{2}$ then $e = x_2$ (as before).

Now return to our unstable network, and add a diagonal spring:

![](../course-assets/images/springs--image07.png)

The new parameter here is the elongation $e_4$.

Now we see that 
$\mathbf{e} = B \mathbf{x}$ where
$$B = \begin{bmatrix} 0 & 1 & 0 & 0 \\
-1 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & \cos(\theta) & \sin(\theta) 
\end{bmatrix}.$$

Let's inspect the force balancing conditions:

![](../course-assets/images/springs--image08.png)

As before, we see that
$$B^T \mathbf{y} = \mathbf{f}$$
so that
$$B^T K B \mathbf{x} = \mathbf{f}.$$

In [82]:
# let's suppose theta = pi/3
theta = np.pi/3
BB = np.array([[0,1,0,0],[-1,0,1,0],[0,0,0,1],[0,0,np.cos(theta),np.sin(theta)]])

K=makeK([1,1,1,1])           ## springs of equal strength
f = np.array([0,1,0,1])    ## downward forces only 

AA = BB.transpose() @ K @ BB

## AA is non-singular!
la.matrix_rank(AA)

4

So there are no *unstable modes* for this system. Non-zero displacements $\mathbf{x}$ are only determined by non-zero force vectors $\mathbf{f}$.

# Least squares and bridges

Suppose we are building a bridge, which we'll view as a spring-system as before:

![](../course-assets/images/springs--image09.png)

For simplicity we are going to assume that all angles in the diagram are $\pi/2$ or $\pi/4$.

Using the same material for each “beam” of the bridge, the spring constants are inversely proportional to their length.

There are 15 springs/beams in total, and we take them as follows:
$$k_i = \left \{ 
\begin{matrix}
9.7500 · 10^6 \operatorname{N/m} & \text{for $i = 1, 2, 3, 4, 6, 9, 12, 14, 15$} \\ 
6.8943 · 10^6 \operatorname{N/m} & \text{for $i = 5, 7, 8, 10, 11, 13$}
\end{matrix}
\right .$$

In order to test the bridge for structural integrity, we load the three lower masses (where traffic
would be) individually with forces of $2 · 10^6$ N, and measure the resulting displacements.

**Remark**
:   The average car in the US weighs about $4, 000$ lbs ($≈ 1, 814$ kg), which
corresponds to $≈ 1.778 · 10^4$ N. 
The average semi-truck weighs about $80, 000$ lbs ($≈ 36,287$ kg),
which corresponds to $≈ 3.556 · 10^5$ N.

In [196]:
# The matrix B with Bx = e is given as follows:
def sbv(i,n):
    return np.array([1 if i == j else 0 for j in range(n)])

def sbvarray(x,l):
    return sbv(l.index(x),len(l))

a = list(range(1,13))

k = np.sqrt(2)/2

B_bridge = np.array([ sbvarray(1,a),                                    # e1 = x1
               sbvarray(3,a) - sbvarray(1,a),                    # e2 = x3 - x1
               sbvarray(5,a) - sbvarray(3,a),                    # e3 = x5 - x3
               sbvarray(5,a),                                    # e4 = x5
               k*(sbvarray(7,a) - sbvarray(8,a)),                # e5 = k*x7 - k*x8
               sbvarray(2,a) - sbvarray(8,a),                    # e6 = x2 - x8
               k*(sbvarray(3,a) + sbvarray(4,a) 
                  - sbvarray(7,a) - sbvarray(8,a)),              # e7 = k*x3 + k*x4 - k*x7 - k*x8 
               k*(-sbvarray(1,a) + sbvarray(2,a)
                  + sbvarray(9,a) - sbvarray(10,a)),             # e8 = -k*x1 + k*x2 + k*x9 - k*x10
               sbvarray(4,a) - sbvarray(10,a),                   # e9 = x4 - x10 
               k*(sbvarray(5,a) + sbvarray(6,a) 
                  - sbvarray(9,a) - sbvarray(10,a)),             # e10 = k*x5 + k*x6 - k*x9 - k*x10
               k*(-sbvarray(3,a) + sbvarray(4,a)
                  + sbvarray(11,a) - sbvarray(12,a)),            # e11 = -k*e3 + k*e4 + k*e11 - k*e12
               sbvarray(6,a) - sbvarray(12,a),                   # e12 = x6 - x12
                  k*(-sbvarray(11,a) - sbvarray(12,a)),          # e13 = -k*x11 + -k*x12
               -sbvarray(7,a) + sbvarray(9,a),                   # e14 = -x7 + x9
               -sbvarray(9,a) + sbvarray(11,a)                   # e15 = -x9 + x11
             ])
B_bridge

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [-1.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , -1.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.70710678, -0.70710678,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  

Let's suppose that $\mathbf{f} = \begin{bmatrix} 0& 2& 0 &2& 0& 2& 0& 0& 0& 0& 0& 0 \end{bmatrix}^T$. (We keep the numbering on the forces is the same as the numbering on the displacements. Thus, these forces
are *downward* on all three masses in the bottom row.)

In [197]:
ff = 10**6*np.array([0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0])

K2 = makeK([ 9.7* 10**6 if i in [1,2,3,4,6,9,12,14,15] else 6.8943 * 10**6 for i in range (1,16)])


A2 = B_bridge.transpose() @ K2 @ B_bridge
la.solve(A2,ff)

array([-2.45008020e-02,  1.48884085e+00, -9.03953583e-18,  1.74923139e+00,
        2.45008020e-02,  1.48884085e+00,  3.63369530e-01,  1.23365368e+00,
        2.77121850e-17,  1.65122818e+00, -3.63369530e-01,  1.23365368e+00])

## inverse problem

The above shows that, given knowledge of the "spring constants" $k_i$, and the applied forces $f_i$, we can estimate the displacements $x_i$. This is the "*forward problem*".

The inverse problem is this: given measurements of the displacements $x_i$, find the spring constants $k_i$.

There are many applications of such *inverse problems*.

For our bridge problem, note that consider the system

$$B^T K B \mathbf{x} = \mathbf{f}$$

We consider a general case where there are $m$ springs and $n/2$ masses, so that there are $n$ "displacement components" (in 2 dimensions).

Notice that

\begin{align*}
K B \mathbf{x} =& \begin{bmatrix} k_1 & 0 & \cdots & 0 \\ 
0 & k_2 & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & k_m \end{bmatrix} \begin{bmatrix} b_{11} & b_{12} & \cdots & b_{1n} \\
b_{21} & b_{22} & \cdots & b_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
b_{m1} & b_{m2} & \cdots & b_{mn} \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \\ \vdots \\ x_n \end{bmatrix} \\
=& \operatorname{diag}(B \mathbf{x}) \begin{bmatrix} k_1 \\ k_2 \\ \vdots \\ k_m \end{bmatrix}
\end{align*}

where for a vector $\mathbf{w} \in \mathbb{R}^m$, we obtain an $m\times m$ matrix
$$\operatorname{diag}(\mathbf{w}) = \begin{bmatrix} w_1 & 0 & \cdots & 0 \\
0 & w_2 & \cdots & 0 \\
\vdots & \vdots& \ddots & \vdots \\
0 & 0 & \cdots & w_m \end{bmatrix}$$

Thus we have
$$B^T K B \mathbf{x} = B^T \operatorname{diag}(B \mathbf{x}) \mathbf{k} = \mathbf{f}$$
which gives some hope for finding the vector $\mathbf{k}$ given knowledge of $\mathbf{x}$ and $\mathbf{f}$.

## the difficulty

Note that in the case of our bridge example, the number of "displacement components" we are tracking is $n = 12$,
while the number of springs is $m = 15$.

Note that $B^T$ is an $n \times m$ matrix, that $B \mathbf{x}$ is in $\mathbb{R}^m$, so that
$B^T \operatorname{diag}(B\mathbf{x})$ is an $n \times m$ matrix.

In our example, the linear equation $B^T \operatorname{diag}(B\mathbf{x}) \mathbf{k} = \mathbf{f}$ amounts to a system of
$12$ linear equations in $15$ unknowns.

In general, if $n < m$, the system is *underdetermined*. As a consequence, the linear equation does not uniquely determine
the spring constants $k_i$ -- i.e. the entries in the vector $\mathbf{k}$.

One way to fix this:

take measurements $\mathbf{x}$ for *various different force loads* $\mathbf{f}$.

More precisely, consider different force loads $\mathbf{f}_1,\cdots,\mathbf{f}_p$.
For each of these force loads, determine the displacement vectors $\mathbf{x}_1,\cdots,\mathbf{x}_p$.

We now obtain a $pn \times m$ matrix system
$$\begin{bmatrix}
B^T \operatorname{diag}(B \mathbf{x}_1) \\
B^T \operatorname{diag}(B \mathbf{x}_2) \\
\vdots \\
B^T \operatorname{diag}(B \mathbf{x}_p) \\
\end{bmatrix} \begin{bmatrix} k_1 \\ k_2 \\ \vdots \\ k_m \end{bmatrix} = 
\begin{bmatrix} \mathbf{f}_1 \\ \mathbf{f}_2 \\ \vdots \\ \mathbf{f}_p \end{bmatrix}.$$

If we choose $p$ sufficiently large and if we make sure that the $np \times m$ coefficient
matrix has rank at least $m$, we expect to find a solution.

In [142]:
def diag(w):
    n = len(w)
    return np.array([ w[i]*sbv(i,n) for i in range(n) ])

diag([1,2,3,4])

array([[1, 0, 0, 0],
       [0, 2, 0, 0],
       [0, 0, 3, 0],
       [0, 0, 0, 4]])

Let's consider some data. We have collected displacement measurements for three different force loads.
The displacement measurements are outside of engineering tolerances!

We want to know which spring (=bridge component) is defective.

In [206]:
# measurements

# for these forces
f1 = 10**6*np.array([0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
f2 = 10**6*np.array([0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0])
f3 = 10**6*np.array([0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0])

# measured the following displacements
x1 = np.array([ 0.04465819,  0.69735727,  0.08117014,  0.49972439,  0.0584346 ,
        0.29520329,  0.09315958,  0.52830165, -0.04706315,  0.50115528,
       -0.11445688,  0.25950424])

x2 =np.array([-0.03453953,  0.49972439,  0.01867222,  0.76169019,  0.03453953,
        0.49283618,  0.16411865,  0.45421336,  0.0034441 ,  0.66577233,
       -0.15233463,  0.44242935])

x3 = np.array([-0.04465819,  0.29520329, -0.08117014,  0.49283618, -0.0584346 ,
        0.69735727,  0.11302599,  0.25807335,  0.04706315,  0.49140529,
       -0.09172868,  0.52687076])

In [208]:
coeffMatrix = np.concatenate([ B_bridge.transpose() @ diag( B_bridge @ x ) for x in [x1,x2,x3] ] )

kvalues,_,_,_ = la.lstsq(coeffMatrix , np.concatenate([f1,f2,f3]),rcond=None)


[ f"k{i+1} = {kvalues[i]}" for i in range(len(kvalues))]

['k1 = 9699998.607722549',
 'k2 = 2000000.4090927793',
 'k3 = 9700000.122052612',
 'k4 = 9699998.86756195',
 'k5 = 6894300.281687661',
 'k6 = 9700000.126240244',
 'k7 = 6894300.249842933',
 'k8 = 6894299.831309714',
 'k9 = 9699998.96686751',
 'k10 = 6894300.258596171',
 'k11 = 6894299.903274672',
 'k12 = 9700000.298712648',
 'k13 = 6894300.008658496',
 'k14 = 9700000.174220743',
 'k15 = 9700000.624519458']

We see that `k2` is different than expected and needs to be replaced!